## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Weather Description,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Port Hardy,50.6996,-127.4199,48.20,broken clouds,93,75,6.91,CA,2021-11-05 04:15:46
1,1,Sembe,1.6481,14.5806,69.21,broken clouds,98,73,0.78,CG,2021-11-05 04:19:56
2,2,Katsuura,35.1333,140.3000,68.04,clear sky,53,2,7.18,JP,2021-11-05 04:19:57
3,3,Vaini,-21.2000,-175.2000,78.96,broken clouds,100,75,8.05,TO,2021-11-05 04:19:57
4,4,Constitucion,-35.3333,-72.4167,51.30,clear sky,61,0,9.46,CL,2021-11-05 04:19:58


In [3]:
# User input for temperature preferences
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 70


In [4]:
#Filter city data based off of user prefences
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Weather Description,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Sembe,1.6481,14.5806,69.21,broken clouds,98,73,0.78,CG,2021-11-05 04:19:56
2,2,Katsuura,35.1333,140.3000,68.04,clear sky,53,2,7.18,JP,2021-11-05 04:19:57
4,4,Constitucion,-35.3333,-72.4167,51.30,clear sky,61,0,9.46,CL,2021-11-05 04:19:58
5,5,Luderitz,-26.6481,15.1594,58.01,few clouds,74,13,13.00,NaN,2021-11-05 04:19:59
7,7,Port Alfred,-33.5906,26.8910,65.64,overcast clouds,83,97,5.64,ZA,2021-11-05 04:16:14
17,17,Rikitea,-23.1203,-134.9692,69.22,moderate rain,89,100,3.00,PF,2021-11-05 04:20:06
20,20,Adeje,28.1227,-16.7260,68.02,broken clouds,72,75,16.11,ES,2021-11-05 04:20:08
21,21,Buraydah,26.3260,43.9750,63.05,clear sky,48,0,8.01,SA,2021-11-05 04:20:08
24,24,Asau,46.4333,26.4000,55.40,broken clouds,60,80,5.06,RO,2021-11-05 04:20:09
35,35,Busselton,-33.6500,115.3333,65.05,broken clouds,61,82,14.58,AU,2021-11-05 04:20:14


In [5]:
preferred_cities_df.count()

City_ID                175
City                   175
Lat                    175
Lng                    175
Max Temp               175
Weather Description    175
Humidity               175
Cloudiness             175
Wind Speed             175
Country                170
Date                   175
dtype: int64

In [6]:
#Drop rows with empty values
clean_df = preferred_cities_df.dropna(axis=0, how="any", thresh=None, subset=None, inplace=False)
clean_df.head()

,City_ID,City,Lat,Lng,Max Temp,Weather Description,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Sembe,1.6481,14.5806,69.21,broken clouds,98,73,0.78,CG,2021-11-05 04:19:56
2,2,Katsuura,35.1333,140.3000,68.04,clear sky,53,2,7.18,JP,2021-11-05 04:19:57
4,4,Constitucion,-35.3333,-72.4167,51.30,clear sky,61,0,9.46,CL,2021-11-05 04:19:58
7,7,Port Alfred,-33.5906,26.8910,65.64,overcast clouds,83,97,5.64,ZA,2021-11-05 04:16:14
17,17,Rikitea,-23.1203,-134.9692,69.22,moderate rain,89,100,3.00,PF,2021-11-05 04:20:06


In [7]:
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Sembe,CG,69.21,broken clouds,1.6481,14.5806,
2,Katsuura,JP,68.04,clear sky,35.1333,140.3000,
4,Constitucion,CL,51.30,clear sky,-35.3333,-72.4167,
7,Port Alfred,ZA,65.64,overcast clouds,-33.5906,26.8910,
17,Rikitea,PF,69.22,moderate rain,-23.1203,-134.9692,


In [8]:
#Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Sembe,CG,69.21,broken clouds,1.6481,14.5806,
2,Katsuura,JP,68.04,clear sky,35.1333,140.3000,Katsuura Hotel Mikazuki
4,Constitucion,CL,51.30,clear sky,-35.3333,-72.4167,Hotel Boutique Las Azucenas
7,Port Alfred,ZA,65.64,overcast clouds,-33.5906,26.8910,The Halyards Hotel
17,Rikitea,PF,69.22,moderate rain,-23.1203,-134.9692,People ThankYou


In [19]:
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna(axis=0, how="any", thresh=None, subset=None, inplace=False)
clean_hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Katsuura,JP,68.04,clear sky,35.1333,140.3000,Katsuura Hotel Mikazuki
4,Constitucion,CL,51.30,clear sky,-35.3333,-72.4167,Hotel Boutique Las Azucenas
7,Port Alfred,ZA,65.64,overcast clouds,-33.5906,26.8910,The Halyards Hotel
17,Rikitea,PF,69.22,moderate rain,-23.1203,-134.9692,People ThankYou
20,Adeje,ES,68.02,broken clouds,28.1227,-16.7260,Gran Tacande Wellness & Relax


In [20]:
output_data_file = 'Vacation_Search/WeatherPy_vacation.csv'
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp}°F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

#Heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
#Marker Layer
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

IndexError: list index out of range